### Импортируем библиотеки

In [1]:
import pandas as pd
import numpy as np
import os
import tqdm
import datetime

In [2]:
import warnings
warnings.filterwarnings("ignore")

### Подгружаем данные

In [3]:
addresses = pd.read_csv('addresses.csv')
user_profiles = pd.read_csv('user_profiles.csv')
shipments_reduced_info = pd.read_csv('shipments_reduced_info.csv')

In [4]:
addresses.head()

,id,phone_id
0,41,1
1,8531,2
2,2982657,2
3,3271801,2
4,3648206,2


In [8]:
user_profiles.head()

,user_id,gender,bdate
0,2226207,NaN,NaN
1,2226207,NaN,NaN
2,2226207,NaN,NaN
3,1684318,male,1987-10-11
4,1684318,male,1987-10-11


In [9]:
shipments_reduced_info.head()

,shipment_id,n_replaced,n_cancelled,n_items_total,total_sum_without_cancelled,total_discounts
0,178163,0,7,31,7489.499998,320.939995
1,273988,4,9,81,33408.880073,2102.400017
2,322307,0,1,24,3529.400031,1198.849987
3,337809,2,0,45,8884.640038,857.299990
4,351762,0,0,54,8297.440020,831.120012


In [15]:
messages.head()

NameError: name 'messages' is not defined

In [17]:
!ls

addresses.csv			 shipments		     user_profiles.xlsx
churn_prediction_baseline.ipynb  shipments_reduced_info.csv
copy				 user_profiles.csv


In [18]:
cd ./shipments

/home/maximus/Yandex.Disk/MyProjects/alivebe/ml_aliexpress/Отток и Uplift/shipments


In [19]:
os.listdir()

['shipments2020-03-01.csv',
 'shipments2020-06-29.csv',
 'shipments2020-04-30.csv',
 'shipments2020-01-01.csv']

### Формируем основной датасет

In [20]:
shipments = pd.DataFrame()
for i in tqdm.tqdm(os.listdir()):
    if 'shipments' in i:
        shipments = pd.concat([shipments, pd.read_csv(i)])
        print(shipments.shape)
    

 25%|██▌       | 1/4 [00:02<00:07,  2.56s/it]

(400963, 20)


 50%|█████     | 2/4 [00:07<00:06,  3.37s/it]

(1273721, 20)


 75%|███████▌  | 3/4 [00:13<00:04,  4.05s/it]

(2119140, 20)


100%|██████████| 4/4 [00:16<00:00,  4.07s/it]

(2260686, 20)


In [21]:
pd.set_option('max_colwidth', 40)

In [22]:
shipments.head()

,user_id,ship_address_id,shipment_id,order_id,order_created_at,order_completed_at,shipment_starts_at,retailer,s.order_state,shipment_state,s.city_name,s.store_id,total_cost,rate,dw_kind,promo_total,total_weight,platform,os,shipped_at
0,4077,4058639,178163,1291559,2017-05-10 10:16:41,2020-03-13 11:57:53,2020-03-15 12:00:00,METRO,canceled,canceled,Москва,1,158.0,0,courier,0.0,27610,web,windows,NaN
1,48861,271117,383088,3015822,2018-02-27 10:48:13,2020-04-24 21:09:15,2020-04-30 07:00:00,METRO,complete,shipped,Москва,3,499.0,0,courier,0.0,87806,web,windows,2020-04-30 12:53:17
2,3119,276189,385477,3019844,2018-03-01 11:58:09,2020-04-25 17:30:52,2020-04-26 12:00:00,METRO,complete,shipped,Москва,14,158.0,0,courier,0.0,23029,web,windows,2020-04-26 11:54:23
3,60931,319599,435693,3095458,2018-04-18 09:45:14,2020-03-15 12:51:48,2020-03-16 20:00:00,METRO,complete,shipped,Москва,8,158.0,5,courier,0.0,8605,app,android,2020-03-16 21:29:10
4,87464,479940,506881,3234709,2018-06-21 12:12:33,2020-03-19 04:43:12,2020-03-25 07:00:00,METRO,complete,shipped,Москва,10,158.0,5,courier,0.0,21323,app,ios,2020-03-25 08:52:31


In [24]:
shipments['phone_id'] = shipments.ship_address_id.map(dict(addresses.values))
shipments = pd.merge(shipments, shipments_reduced_info, on = 'shipment_id', how = 'left')

###  Переводим ts фичи - формируем всевозможные временные дельты

In [25]:
shipments['order_created_at'] = pd.to_datetime(shipments['order_created_at'], format='%Y-%m-%d %H:%M:%S')
shipments['shipment_starts_at'] = pd.to_datetime(shipments['shipment_starts_at'], format='%Y-%m-%d %H:%M:%S')
shipments['shipped_at'] = pd.to_datetime(shipments['shipped_at'], format='%Y-%m-%d %H:%M:%S')
shipments['order_completed_at'] = pd.to_datetime(shipments['order_completed_at'], format='%Y-%m-%d %H:%M:%S')


shipments['timediff_1'] = (shipments['order_created_at']-shipments['shipment_starts_at']).astype('timedelta64[h]')
shipments['timediff_2'] = (shipments['order_created_at']-shipments['shipped_at']).astype('timedelta64[h]')
shipments['timediff_3'] = (shipments['shipment_starts_at']-shipments['shipped_at']).astype('timedelta64[h]')
shipments['timediff_4'] = (shipments['order_created_at']-shipments['order_completed_at']).astype('timedelta64[h]')
shipments['timediff_5'] = (shipments['shipment_starts_at']-shipments['order_completed_at']).astype('timedelta64[h]')
shipments['timediff_6'] = (shipments['shipped_at']-shipments['order_completed_at']).astype('timedelta64[h]')



time_features = ['order_created_at', 'shipment_starts_at', 'shipped_at', 'order_completed_at']

for i in tqdm.tqdm(time_features):

        shipments[i+'_day'] = [x.day for x in shipments[i]]
        shipments[i+'_year'] = [x.year for x in shipments[i]]
        shipments[i+'_month'] = [x.month for x in shipments[i]]
    

100%|██████████| 4/4 [01:10<00:00, 17.64s/it]


0         -24962.0
1         -19029.0
2         -18889.0
3         -16763.0
4         -15427.0
            ...   
2260681      -11.0
2260682      -20.0
2260683      -15.0
2260684      -19.0
2260685     -173.0
Name: timediff_1, Length: 2260686, dtype: float64

### Feature engineering

In [26]:
#
#исправьте валидацию
#

for val_month, val_first_date in tqdm.tqdm(zip([8], ['2020-08-01'])):


    train1 = pd.DataFrame(columns = ['phone_id', 'label'])
    train0 = pd.DataFrame(columns = ['phone_id', 'label'])

    train1['phone_id'] = shipments[shipments['order_completed_at_month'] == val_month].phone_id.drop_duplicates().values
    train1['label'] = 1#были продажи

    train0['phone_id'] = list(set(shipments['phone_id']) - set(train1['phone_id']))
    train0['label'] = 0#не было продаж

    train = pd.concat([train1, train0])


    shipments_train = shipments[shipments['order_completed_at_month'] < val_month]

    for num_day in tqdm.tqdm([365, 180, 120, 90, 60, 30, 15, 7]):

        shipments_train_tmp = shipments_train[shipments_train['order_completed_at'] > np.datetime64(val_first_date) - np.timedelta64(num_day,'D')]# проверить
 

        shipments_train_tmp['timediff_1'+str(num_day)] = shipments_train_tmp.groupby(['phone_id'])['timediff_1'].transform('mean')
        shipments_train_tmp['timediff_2'+str(num_day)] = shipments_train_tmp.groupby(['phone_id'])['timediff_2'].transform('mean')
        shipments_train_tmp['timediff_3'+str(num_day)] = shipments_train_tmp.groupby(['phone_id'])['timediff_3'].transform('mean')
        shipments_train_tmp['timediff_4'+str(num_day)] = shipments_train_tmp.groupby(['phone_id'])['timediff_4'].transform('mean')
        shipments_train_tmp['timediff_5'+str(num_day)] = shipments_train_tmp.groupby(['phone_id'])['timediff_5'].transform('mean')
        shipments_train_tmp['timediff_6'+str(num_day)] = shipments_train_tmp.groupby(['phone_id'])['timediff_6'].transform('mean')
        
        
        #
        #добавьте сюда фичи
        #
        
        shipments_train_tmp = shipments_train_tmp[['phone_id',
                                                   'timediff_1'+str(num_day),
                                                   'timediff_2'+str(num_day),
                                                   'timediff_3'+str(num_day),
                                                   'timediff_4'+str(num_day),
                                                   'timediff_5'+str(num_day),
                                                   'timediff_6'+str(num_day)
                                                  ]].drop_duplicates()
        
        shipments_train_tmp = shipments_train_tmp.groupby('phone_id').max().reset_index()
        

        train = pd.merge(train, shipments_train_tmp, on = 'phone_id', how = 'left')
        train = train.fillna(0)
        
    train['val_month'] = val_month
        
         
          

0it [00:00, ?it/s]
100%|██████████| 8/8 [01:29<00:00, 11.22s/it]
1it [01:43, 103.58s/it]


### Обучаем модель

In [28]:
!pip install lightgbm

     |████████████████████████████████| 1.8MB 71kB/s eta 0:00:011


In [29]:
import lightgbm as lgb

In [36]:
train

,phone_id,label,timediff_1365,timediff_2365,timediff_3365,timediff_4365,timediff_5365,timediff_6365,timediff_1180,timediff_2180,...,timediff_415,timediff_515,timediff_615,timediff_17,timediff_27,timediff_37,timediff_47,timediff_57,timediff_67,val_month
0,197574,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8
1,122345,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8
2,232926,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8
3,236934,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8
4,260786,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606578,717478,0,-119.0,-121.0,-2.0,-117.0,1.0,3.0,-119.0,-121.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8
606579,717480,0,-25.0,-28.0,-4.0,-19.0,6.0,9.0,-25.0,-28.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8
606580,717481,0,-4.0,-4.0,-1.0,-2.0,1.0,2.0,-4.0,-4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8
606581,717482,0,-992.0,-994.0,-3.0,-989.0,3.0,5.0,-992.0,-994.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8


In [ ]:
train = train[train.val_month == 8]

In [ ]:
#
#потюнить параметры
#

import lightgbm as lgb

params = {
    'objective': 'binary',
    'learning_rate': 0.01,
    #'num_leaves': 80,
    'random_seed':666,
     #'num_leaves': 50, 
     #'bagging_freq': 5, 
     #'min_child_samples': 23, 
     #'min_data_in_leaf': 20,  
     #'feature_fraction': 0.4, 
     #'bagging_fraction': 0.9, 
     #'min_sum_hessian_in_leaf': 10,
     #'max_depth' : 8
    
}


In [ ]:
pred_list = []
true_list = []
val_list = []
X = train.drop(['label'], axis = 1)
y = train.label

In [ ]:
from sklearn.model_selection import train_test_split


for rs in [42]:

    X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.33, random_state=rs)
    
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)


    model = lgb.train(
        params, lgb_train,
        valid_sets=[lgb_train, lgb_eval],
        verbose_eval=100,
        num_boost_round=20000,
        early_stopping_rounds=100
    )

    pred_list.append(model.predict(test[X_train.columns]))
    true_list.append(y_test)
    val_list.append(model.predict(X_test[X_train.columns]))


[LightGBM] [Info] Number of positive: 146561, number of negative: 259849
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.070094 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11579
[LightGBM] [Info] Number of data points in the train set: 406410, number of used features: 49
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.360624 -> initscore=-0.572659
[LightGBM] [Info] Start training from score -0.572659
Training until validation scores don't improve for 100 rounds
[100]	training's binary_logloss: 0.409258	valid_1's binary_logloss: 0.408542
[200]	training's binary_logloss: 0.35317	valid_1's binary_logloss: 0.352539
[300]	training's binary_logloss: 0.336712	valid_1's binary_logloss: 0.336403
[400]	training's binary_logloss: 0.330856	valid_1's binary_logloss: 0.330944
[500]	training's binary_logloss: 0.328175	valid_1's binary_logloss: 0.32876
[600]	training's binary_logloss: 0.326641	valid_1's bi

In [ ]:
i = 0
f1_score(true_list[i], val_list[i]>0.42)

0.7965099874839933